In [92]:
import pandas as pd
import os

In [93]:
caminho_raw = "../dados/raw"

In [94]:
df_dre = pd.read_parquet(os.path.join(caminho_raw,'dre.parquet'))

In [95]:
colunas_renomeadas = {
    'Instituição': 'instituicao',
    'Código': 'codigo_bacen',
    'Conglomerado Prudencial': 'conglomerado',
    'TCB': 'tcb',
    'TC': 'tc',
    'TI': 'ti',
    'Cidade': 'cidade',
    'UF': 'uf',
    'Data': 'data_base',
    'Resultado de Intermediação Financeira': 'resultado_intermediacao',
    'Resultado com Transações de Pagamento': 'resultado_pagamentos',
    'Outras Receitas / Despesas': 'outras_receitas_despesas',
    'Resultado antes da Tributação e Participações (x) = (k) + (l) + (w)': 'resultado_antes_impostos', # LAIR
    'Imposto de Renda e Contribuição Social (y)': 'impostos_ir_csll',
    'Participações no Lucro (z)': 'participacoes_lucro',
    'Lucro Líquido (aa) = (x) + (y) + (z)': 'lucro_liquido'
}

for col_antiga, col_nova in colunas_renomeadas.items():
    df_dre = df_dre.rename(columns=colunas_renomeadas)

In [96]:
colunas_para_float = [
    'resultado_intermediacao',
    'resultado_pagamentos',
    'outras_receitas_despesas',
    'impostos_ir_csll',
    'participacoes_lucro',
    'resultado_antes_impostos',
    'lucro_liquido'
]

colunas_para_int = ['codigo_bacen', 'tc', 'ti']

for col in colunas_para_float:
    df_dre[col] = df_dre[col].astype(str).str.replace('.', '', regex=False)
    df_dre[col] = df_dre[col].str.replace(',', '.', regex=False)
    df_dre[col] = pd.to_numeric(df_dre[col], errors='coerce')

for col in colunas_para_int:
    df_dre[col] = df_dre[col].astype(object)

df_dre['data_base'] = pd.to_datetime(df_dre['data_base'])

df_dre.info()
df_dre.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17165 entries, 0 to 17164
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   instituicao               17165 non-null  object        
 1   codigo_bacen              17165 non-null  object        
 2   conglomerado              8361 non-null   object        
 3   tcb                       17165 non-null  object        
 4   tc                        17165 non-null  object        
 5   ti                        17165 non-null  object        
 6   cidade                    17165 non-null  object        
 7   uf                        17165 non-null  object        
 8   data_base                 17165 non-null  datetime64[ns]
 9   resultado_intermediacao   15476 non-null  float64       
 10  resultado_pagamentos      17081 non-null  float64       
 11  outras_receitas_despesas  15476 non-null  float64       
 12  resultado_antes_im

C:\Users\Thiag\AppData\Local\Temp\ipykernel_15820\1290000720.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_dre['data_base'] = pd.to_datetime(df_dre['data_base'])


,instituicao,codigo_bacen,conglomerado,tcb,tc,ti,cidade,uf,data_base,resultado_intermediacao,resultado_pagamentos,outras_receitas_despesas,resultado_antes_impostos,impostos_ir_csll,participacoes_lucro,lucro_liquido
0,BANCO DO BRASIL S.A.,0.0,BB - PRUDENCIAL,b1,1.0,1.0,BRASILIA,DF,2022-12-01,58297195.0,0.0,6287062.0,21753526.0,-2982566.0,-2132589.0,16638371.0
1,CAIXA ECONOMICA FEDERAL,360305.0,CAIXA ECONÔMICA FEDERAL - PRUDENCIAL,b1,1.0,6.0,BRASILIA,DF,2022-12-01,56235680.0,0.0,8480850.0,5582178.0,971396.0,-1060139.0,5493435.0
2,ITAÚ UNIBANCO S.A.,60701190.0,ITAU - PRUDENCIAL,b1,2.0,8.0,SAO PAULO,SP,2022-12-01,36162795.0,0.0,4631069.0,18299359.0,1334900.0,-3690333.0,15943927.0
3,BANCO BRADESCO S.A.,60746948.0,BRADESCO - PRUDENCIAL,b1,2.0,8.0,OSASCO,SP,2022-12-01,44706379.0,0.0,6034638.0,2897870.0,3749785.0,0.0,6647656.0
4,BANCO SANTANDER (BRASIL) S.A.,90400888.0,SANTANDER - PRUDENCIAL,b1,3.0,8.0,SAO PAULO,SP,2022-12-01,26385861.0,0.0,5637941.0,2602071.0,2970185.0,-990117.0,4582139.0
